# TCP

Es necesario importar el JSON con las tiendas y coordenadas para introducirlo al codigo y funcione de forma adecuada.

# Leer informacion de archivo

Las tiendas y coordenadas son extraídos del archivo Json.

In [ ]:
import json

# Read capital names and coordinates from json file
try:
    with open('TuArchivoJson.json', 'r', encoding='utf-8') as file:
        capitals_json = json.load(file)
except UnicodeDecodeError:
    with open('TuArchivoJson.json', 'r', encoding='latin1') as file:
        capitals_json = json.load(file)

# Lists to store extracted data
capitals = []
coordinates = {}

# Iterate over each item in the JSON list and extract data
for item in capitals_json:
    capital = item['Tienda']
    latitud = item['Latitud']
    longitud = item['Longitud']

    capitals.append(capital)
    coordinates[capital] = (float(latitud), float(longitud))

# Computacion de datos

La siguiente funcion calcula la distancia para cada par de tiendas. Aqui estamos usando combinaciones de tiendas.

import math
from itertools import combinations

# Compute pairwise distance matrix

def distance(city1, city2):
    c1 = coordinates[city1]
    c2 = coordinates[city2]
    diff = (c1[0]-c2[0], c1[1]-c2[1])
    return math.sqrt(diff[0]*diff[0]+diff[1]*diff[1])

dist = {(c1, c2): distance(c1, c2) for c1, c2 in combinations(capitals, 2)}

# Pares de tiendas

Combina las tiendas en orden para seguir la ruta

In [ ]:
def print_consecutive_distances(cities):
    for i in range(len(cities) - 1):
        city1 = cities[i]
        city2 = cities[i + 1]
        dist = distance(city1, city2)
        print(f"{city1} -> {city2}: {dist:.6f}")

# Example usage
print_consecutive_distances(capitals)

# Codigo del modelo

Ahora se escribe el modelo TSP, definiendo las variables de decision, restricciones y funcion objetivo.

Nota: como estamos usando combinaciones, establecemos el objeto x[j,i] a x[i,j] en vez de una restriccion.

In [ ]:
import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.7 & Gurobi 9.0.0

m = gp.Model()

# Variables: is city 'i' adjacent to city 'j' on the tour?
vars = m.addVars(dist.keys(), obj=dist, vtype=GRB.BINARY, name='x')

# Symmetric direction: use dict.update to alias variable with new key
vars.update({(j,i):vars[i,j] for i,j in vars.keys()})

# Constraints: two edges incident to each city
cons = m.addConstrs(vars.sum(c, '*') == 2 for c in capitals)

# Definicion de devolucion de llamada

Las restricciones de subtour evitan múltiples bucles en un tour TSP. Como hay un numero exponencial de restricciones, no los agregaremos al modelo.

Optamos usar una funcion de devolucion de llamada para buscar restricciones violadas y agregarlos al modelo como restricciones diferidas.

In [ ]:
# Callback - use lazy constraints to eliminate sub-tours

def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        # make a list of edges selected in the solution
        vals = model.cbGetSolution(model._vars)
        selected = gp.tuplelist((i, j) for i, j in model._vars.keys()
                             if vals[i, j] > 0.5)
        # find the shortest cycle in the selected edge list
        tour = subtour(selected)
        if len(tour) < len(capitals):
            # add subtour elimination constr. for every pair of cities in subtour
            model.cbLazy(gp.quicksum(model._vars[i, j] for i, j in combinations(tour, 2))
                         <= len(tour)-1)

# Given a tuplelist of edges, find the shortest subtour

def subtour(edges):
    unvisited = capitals[:]
    cycle = capitals[:] # Dummy - guaranteed to be replaced
    while unvisited:  # true if list is non-empty
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*')
                         if j in unvisited]
        if len(thiscycle) <= len(cycle):
            cycle = thiscycle # New shortest subtour
    return cycle

# Solucionar el modelo

In [ ]:
m._vars = vars
m.Params.lazyConstraints = 1
m.optimize(subtourelim)

# Analisis

Recuperamos la solucion optima del TSP y verificamos que la ruta optima va a todas las tiendas y regresa a la tienda de partida.

In [ ]:
# Retrieve solution

vals = m.getAttr('x', vars)
selected = gp.tuplelist((i, j) for i, j in vals.keys() if vals[i, j] > 0.5)

tour = subtour(selected)
assert len(tour) == len(capitals)

La ruta optima se desplega en el mapa.

In [ ]:
# Map the solution

import folium

map = folium.Map(location=[40,-95], zoom_start = 4)

points = []
for city in tour:
  points.append(coordinates[city])
points.append(points[0])

folium.PolyLine(points).add_to(map)

map